# Building Tree Clusters Pipeline

This code will build the supervised tree clusters to minimize distance between samples in each node. In order to run this code you will need to: 
1. Create your own pre_process function to read in your dataset and return the following dataframes: xTrain, xTest, yTrain, yTest, df_train, df_test (the last 2 are the full dataframes for training and testing) 
2. Run all cells below 
3. Call get_tree_clust(depth, pre_process_func) on the depth you want for your tree and the pre_process function that you just created. Your resulting tree will be printed and we will return the ROC AUC score for the tree and the resulting tree itself. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import scipy.stats as sc
# import shap
# import lime
import sklearn 
import warnings
#import xgboost
import itertools
import gc
import networkx as nx
import pydot

import multiprocessing as mp
from itertools import product 
import time
import pickle 
#import interpret
import collections
import math
import random
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import  RandomForestRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn import preprocessing
# from interpret.glassbox import ExplainableBoostingClassifier
# from interpret import show
from sklearn.preprocessing import LabelEncoder
# from interpret.glassbox import ExplainableBoostingRegressor
# from interpret import show
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import AdaBoostRegressor
%matplotlib inline
import os, sys
#import statsmodels.api as sm
sys.path.append(os.path.abspath("../../../"))

pd.set_option('display.max_columns', None)
import eli5
from eli5.sklearn import PermutationImportance

import sys
print(sys.path)

sys.path.append("/anaconda3/lib/python3.6/site-packages")
 

['/Users/aparnacalambur/Documents/Cornell/Classes/MEng/interpretable_clustering', '', '/Library/Python/3.6/site-packages', '/Users/aparnacalambur/Documents/Cornell/Classes/MEng/interpretable_clustering', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '/Users/aparnacalambur/Library/Python/3.7/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions', '/Users/aparnacalambur/.ipython', '/Users/aparnacalambur/Documents/Cornell']


In [2]:
# preprocessing
def pre_process(ts=0.3):
    titanic = pd.read_csv('train.csv')
    #titanic
    full_data = titanic
    full_data = full_data.drop(['PassengerId'], axis=1)

    deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
    full_data['Cabin'] = full_data['Cabin'].fillna("U0")
    full_data['Deck'] = full_data['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    full_data['Deck'] = full_data['Deck'].map(deck)
    full_data['Deck'] = full_data['Deck'].fillna(0)
    full_data['Deck'] = full_data['Deck'].astype(int)

    full_data = full_data.drop('Cabin', axis = 1)

    mean = full_data["Age"].mean()
    std = full_data["Age"].std()
    is_null = full_data["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = full_data["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    full_data["Age"] = age_slice
    full_data["Age"] = full_data["Age"].astype(int)
    full_data["Age"].isnull().sum()
    full_data['Embarked'] = full_data['Embarked'].fillna('S')


    full_data['Fare'] = full_data['Fare'].fillna(0)
    full_data['Fare'] = full_data['Fare'].astype(int)
    full_data = full_data.drop(['Name'], axis=1)
    full_data = full_data.drop(['Ticket'], axis=1)
    full_data['Sex'] = full_data['Sex'].map({"male": 0, "female": 1})


    full_data_train,full_data_test = train_test_split(full_data,test_size = ts,random_state = 10)

    full_data = pd.get_dummies(full_data,columns = ['Pclass','Embarked','Deck'])
    full_data = full_data.dropna().reset_index().drop('index',axis = 1)
    X = full_data.drop('Survived',axis = 1)
    cols = X.columns
    y = full_data['Survived']
    scaler = preprocessing.StandardScaler()
    X = scaler.fit_transform(X)
    X =pd.DataFrame(X, columns = cols)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = ts,random_state = 10)
    full_data_train = full_data_train[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Deck"]]
    full_data_test = full_data_test[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Deck"]]
    return xTrain, xTest, yTrain, yTest, full_data_train, full_data_test


In [3]:
def pre_proc_synthetic_non_linear():
    n = 300
    X = np.random.randn(n, 10)
    y = -2 * np.sin(2*X[:,0]*X[:,2] ) + np.maximum(X[:,1], 0)  + np.exp(-X[:,3]) + np.random.randn(n)
    cols = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
    X = preprocessing.scale(X)
    y = preprocessing.scale(y)
    X = pd.DataFrame(X,columns = cols)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)
    c = ['y'] + cols
    full_data = pd.DataFrame(columns=c)
    full_data['y'] = y
    for col in cols:
        full_data[col] = X[col]
    
    full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)
    return xTrain, xTest, yTrain, yTest, full_data_train, full_data_test

In [4]:
def pre_processing_adult():
    income_data = pd.read_csv('adult.csv')
    inc_no_tar = income_data.drop(columns = ['income'])
    c = ['income'] + list(inc_no_tar.columns)
    data = pd.DataFrame(columns=c)
    for col in c:
        data[col] = income_data[col]
    income_data = data.copy()

    income_data['workclass']= income_data['workclass'].replace({"?":"Unknown"})
    income_data['native-country']= income_data['native-country'].replace({"?":"Unknown"})
    income_data = income_data.drop(['education', 'marital-status', 'fnlwgt'], axis=1)
    
    label_encoder = preprocessing.LabelEncoder()
    income_data['workclass']= label_encoder.fit_transform(income_data['workclass'])
    income_data['occupation']= label_encoder.fit_transform(income_data['occupation'])
    income_data['relationship']= label_encoder.fit_transform(income_data['relationship'])
    income_data['race']= label_encoder.fit_transform(income_data['race'])
    income_data['gender']= label_encoder.fit_transform(income_data['gender'])
    income_data['native-country']= label_encoder.fit_transform(income_data['native-country'])
    income_data['income']= label_encoder.fit_transform(income_data['income'])
    
    full_data = income_data.sample(n=1000, random_state=1)

    full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)

    full_data = full_data.dropna().reset_index().drop('index',axis = 1)
    X = full_data.drop('income',axis = 1)
    cols = X.columns
    y = full_data['income']
    scaler = preprocessing.StandardScaler()
    X = scaler.fit_transform(X)
    X =pd.DataFrame(X, columns = cols)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)
    return xTrain, xTest, yTrain, yTest, full_data_train,full_data_test  

In [5]:
def pre_processing_adult2():
    income_data = pd.read_csv('adult.csv')
    inc_no_tar = income_data.drop(columns = ['income'])
    c = ['income'] + list(inc_no_tar.columns)
    data = pd.DataFrame(columns=c)
    for col in c:
        data[col] = income_data[col]
    income_data = data.copy()

    income_data['workclass']= income_data['workclass'].replace({"?":"Unknown"})
    income_data['native-country']= income_data['native-country'].replace({"?":"Unknown"})
    income_data = income_data.drop(['education', 'marital-status', 'fnlwgt'], axis=1)
    
    label_encoder = preprocessing.LabelEncoder()
    income_data['workclass']= label_encoder.fit_transform(income_data['workclass'])
    income_data['occupation']= label_encoder.fit_transform(income_data['occupation'])
    income_data['relationship']= label_encoder.fit_transform(income_data['relationship'])
    income_data['race']= label_encoder.fit_transform(income_data['race'])
    income_data['gender']= label_encoder.fit_transform(income_data['gender'])
    income_data['native-country']= label_encoder.fit_transform(income_data['native-country'])
    income_data['income']= label_encoder.fit_transform(income_data['income'])
    
    full_data = income_data.sample(n=2500, random_state=1)

    full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)

    full_data = full_data.dropna().reset_index().drop('index',axis = 1)
    X = full_data.drop('income',axis = 1)
    cols = X.columns
    y = full_data['income']
    scaler = preprocessing.StandardScaler()
    X = scaler.fit_transform(X)
    X =pd.DataFrame(X, columns = cols)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)
    return xTrain, xTest, yTrain, yTest, full_data_train,full_data_test  

In [6]:
def pre_processing_adult3():
    income_data = pd.read_csv('adult.csv')
    inc_no_tar = income_data.drop(columns = ['income'])
    c = ['income'] + list(inc_no_tar.columns)
    data = pd.DataFrame(columns=c)
    for col in c:
        data[col] = income_data[col]
    income_data = data.copy()

    income_data['workclass']= income_data['workclass'].replace({"?":"Unknown"})
    income_data['native-country']= income_data['native-country'].replace({"?":"Unknown"})
    income_data = income_data.drop(['education', 'marital-status', 'fnlwgt'], axis=1)
    
    label_encoder = preprocessing.LabelEncoder()
    income_data['workclass']= label_encoder.fit_transform(income_data['workclass'])
    income_data['occupation']= label_encoder.fit_transform(income_data['occupation'])
    income_data['relationship']= label_encoder.fit_transform(income_data['relationship'])
    income_data['race']= label_encoder.fit_transform(income_data['race'])
    income_data['gender']= label_encoder.fit_transform(income_data['gender'])
    income_data['native-country']= label_encoder.fit_transform(income_data['native-country'])
    income_data['income']= label_encoder.fit_transform(income_data['income'])
    
    full_data = income_data.sample(n=5000, random_state=1)

    full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)

    full_data = full_data.dropna().reset_index().drop('index',axis = 1)
    X = full_data.drop('income',axis = 1)
    cols = X.columns
    y = full_data['income']
    scaler = preprocessing.StandardScaler()
    X = scaler.fit_transform(X)
    X =pd.DataFrame(X, columns = cols)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)
    return xTrain, xTest, yTrain, yTest, full_data_train,full_data_test  

In [7]:
def pre_processing_adult4():
    income_data = pd.read_csv('adult.csv')
    inc_no_tar = income_data.drop(columns = ['income'])
    c = ['income'] + list(inc_no_tar.columns)
    data = pd.DataFrame(columns=c)
    for col in c:
        data[col] = income_data[col]
    income_data = data.copy()

    income_data['workclass']= income_data['workclass'].replace({"?":"Unknown"})
    income_data['native-country']= income_data['native-country'].replace({"?":"Unknown"})
    income_data = income_data.drop(['education', 'marital-status', 'fnlwgt'], axis=1)
    
    label_encoder = preprocessing.LabelEncoder()
    income_data['workclass']= label_encoder.fit_transform(income_data['workclass'])
    income_data['occupation']= label_encoder.fit_transform(income_data['occupation'])
    income_data['relationship']= label_encoder.fit_transform(income_data['relationship'])
    income_data['race']= label_encoder.fit_transform(income_data['race'])
    income_data['gender']= label_encoder.fit_transform(income_data['gender'])
    income_data['native-country']= label_encoder.fit_transform(income_data['native-country'])
    income_data['income']= label_encoder.fit_transform(income_data['income'])
    
    full_data = income_data.sample(n=10000, random_state=1)

    full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)

    full_data = full_data.dropna().reset_index().drop('index',axis = 1)
    X = full_data.drop('income',axis = 1)
    cols = X.columns
    y = full_data['income']
    scaler = preprocessing.StandardScaler()
    X = scaler.fit_transform(X)
    X =pd.DataFrame(X, columns = cols)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)
    return xTrain, xTest, yTrain, yTest, full_data_train,full_data_test  

In [8]:
def pre_processing_adult5():
    income_data = pd.read_csv('adult.csv')
    inc_no_tar = income_data.drop(columns = ['income'])
    c = ['income'] + list(inc_no_tar.columns)
    data = pd.DataFrame(columns=c)
    for col in c:
        data[col] = income_data[col]
    income_data = data.copy()

    income_data['workclass']= income_data['workclass'].replace({"?":"Unknown"})
    income_data['native-country']= income_data['native-country'].replace({"?":"Unknown"})
    income_data = income_data.drop(['education', 'marital-status', 'fnlwgt'], axis=1)
    
    label_encoder = preprocessing.LabelEncoder()
    income_data['workclass']= label_encoder.fit_transform(income_data['workclass'])
    income_data['occupation']= label_encoder.fit_transform(income_data['occupation'])
    income_data['relationship']= label_encoder.fit_transform(income_data['relationship'])
    income_data['race']= label_encoder.fit_transform(income_data['race'])
    income_data['gender']= label_encoder.fit_transform(income_data['gender'])
    income_data['native-country']= label_encoder.fit_transform(income_data['native-country'])
    income_data['income']= label_encoder.fit_transform(income_data['income'])
    
    full_data = income_data.sample(n=20000, random_state=1)

    full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)

    full_data = full_data.dropna().reset_index().drop('index',axis = 1)
    X = full_data.drop('income',axis = 1)
    cols = X.columns
    y = full_data['income']
    scaler = preprocessing.StandardScaler()
    X = scaler.fit_transform(X)
    X =pd.DataFrame(X, columns = cols)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)
    return xTrain, xTest, yTrain, yTest, full_data_train,full_data_test  

In [9]:
def pre_processing_adult6():
    income_data = pd.read_csv('adult.csv')
    inc_no_tar = income_data.drop(columns = ['income'])
    c = ['income'] + list(inc_no_tar.columns)
    data = pd.DataFrame(columns=c)
    for col in c:
        data[col] = income_data[col]
    income_data = data.copy()

    income_data['workclass']= income_data['workclass'].replace({"?":"Unknown"})
    income_data['native-country']= income_data['native-country'].replace({"?":"Unknown"})
    income_data = income_data.drop(['education', 'marital-status', 'fnlwgt'], axis=1)
    
    label_encoder = preprocessing.LabelEncoder()
    income_data['workclass']= label_encoder.fit_transform(income_data['workclass'])
    income_data['occupation']= label_encoder.fit_transform(income_data['occupation'])
    income_data['relationship']= label_encoder.fit_transform(income_data['relationship'])
    income_data['race']= label_encoder.fit_transform(income_data['race'])
    income_data['gender']= label_encoder.fit_transform(income_data['gender'])
    income_data['native-country']= label_encoder.fit_transform(income_data['native-country'])
    income_data['income']= label_encoder.fit_transform(income_data['income'])
    
    full_data = income_data.sample(n=40000, random_state=1)

    full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)

    full_data = full_data.dropna().reset_index().drop('index',axis = 1)
    X = full_data.drop('income',axis = 1)
    cols = X.columns
    y = full_data['income']
    scaler = preprocessing.StandardScaler()
    X = scaler.fit_transform(X)
    X =pd.DataFrame(X, columns = cols)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)
    return xTrain, xTest, yTrain, yTest, full_data_train,full_data_test  

In [10]:
def pre_processing_heart():
    heart = pd.read_csv("heart.csv")
    #full_data = heart
    heart = pd.read_csv("heart.csv")
    heart_no_tar = heart.drop(columns = ['target'])
    c = ['target'] + list(heart_no_tar.columns)
    full_data = pd.DataFrame(columns=c)
    for col in c:
        full_data[col] = heart[col]

    full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)

    full_data = full_data.dropna().reset_index().drop('index',axis = 1)
    X = full_data.drop('target',axis = 1)
    cols = X.columns
    y = full_data['target']
    scaler = preprocessing.StandardScaler()
    X = scaler.fit_transform(X)
    X =pd.DataFrame(X, columns = cols)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)
    return xTrain, xTest, yTrain, yTest, full_data_train,full_data_test  

https://scikit-learn.org/dev/auto_examples/tree/plot_unveil_tree_structure.html

# Distance Calc

In [11]:
# def floyd_warshall(G):
#     nV = len(G)
#     distance = list(map(lambda i: list(map(lambda j: j, i)), G))
#     n = 0

#     # Adding vertices individually
#     for k in range(nV):
#         for i in range(nV):
#             for j in range(nV):
#                 distance[i][j] = min(distance[i][j], distance[i][k] + distance[k][j])
#                 n += 1
#     return distance, n

In [12]:
# def get_tree_dists(estimator):
#     n_nodes = estimator.tree_.node_count
#     children_left = estimator.tree_.children_left
#     children_right = estimator.tree_.children_right

#     dists = np.empty((n_nodes,n_nodes)); dists.fill(10000)
#     for i in range(len(children_left)):
#         left_node_id = children_left[i]
#         if left_node_id != -1:
#             dists[i][left_node_id] = 1
#             dists[left_node_id][i] = 1

#     for i in range(len(children_right)):
#         if children_right[i] != -1:
#             dists[i][children_right[i]] = 1
#             dists[children_right[i]][i] = 1
#     return dists

In [13]:
# def build_tree_fw(xTrain,yTrain,xTest, md):
#     train = xTrain.copy()
#     train['y'] = yTrain
#     train1 = train.sample(n = len(train), replace = True) 
#     train1=train1.dropna(how='any')
#     yTrain1 = train1['y']
#     xTrain1 = train1.drop('y',axis = 1)
#     gc.collect()
    
#     estimator = DecisionTreeClassifier(max_depth=md).fit(xTrain1,yTrain1)
#     leaves_train = estimator.apply(xTrain)
#     leaves_test = estimator.apply(xTest)
#     train_comb = list(itertools.combinations(range(0,len(xTrain)), 2))
#     #test_comb = list(itertools.combinations(range(0,len(xTest)), 2))
#     graph = get_tree_dists(estimator)
#     fw_dist, n = floyd_warshall(graph)
    
#     i1_train = [i for i, _ in train_comb]
#     i2_train = [i for _, i in train_comb]
#     train_dists = [fw_dist[leaves_train[i]][leaves_train[j]] for i, j in train_comb]
#     train_dist_df = pd.DataFrame(i1_train,columns=['i1'])
#     train_dist_df['i2'] = i2_train
#     train_dist_df['tree_dist'] = train_dists
    
# #     i1_test = [i for i, _ in test_comb]
# #     i2_test = [i for _, i in test_comb]
# #     test_dists = [fw_dist[leaves_test[i]][leaves_test[j]] for i, j in test_comb]
# #     test_dist_df = pd.DataFrame(i1_test,columns=['i1'])
# #     test_dist_df['i2'] = i2_test
# #     test_dist_df['tree_dist'] = test_dists
    
#     return([estimator,train_dist_df, n])

In [14]:
# def fit_random_forest(xTrain,yTrain,num_trees,xTest, md):
#     i = 0
#     mods = []
#     nodes = 0
#     train_dists = pd.DataFrame()
#    # test_dists = pd.DataFrame()
#     while i <= num_trees:
#         #print(yTrain)
#         tree = build_tree_fw(xTrain,yTrain,xTest, md)
#         mods.append(tree[0])
#         train_dists = train_dists.append(tree[1])
#         nodes += (tree[2])
#         #test_dists = test_dists.append(tree[2])
#         i = i+1
        
#     train_final_dist = train_dists.groupby(['i1','i2']).mean().reset_index()
#    # test_final_dist = test_dists.groupby(['i1','i2']).mean().reset_index()
#     return(mods,train_final_dist, nodes, nodes/100)

In [15]:
# def rf_predict(xTest,mods):
#     pred = []
#     for clf in mods:
#         pred.append(clf.predict(xTest))
#     pred = np.mean(pred,axis = 0)
#     pred = [int(x) for x in pred>=0.5]
#     return pred


In [16]:
def compute_distances(tree1, X):
    n_nodes = tree1.tree_.node_count
    children_left = tree1.tree_.children_left
    children_right = tree1.tree_.children_right
    feature = tree1.tree_.feature
    threshold = tree1.tree_.threshold

    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
    while len(stack) > 0:
        node_id, depth = stack.pop()
        node_depth[node_id] = depth
        is_split_node = children_left[node_id] != children_right[node_id]
        if is_split_node:
            stack.append((children_left[node_id], depth + 1))
            stack.append((children_right[node_id], depth + 1))
        else:
            is_leaves[node_id] = True

    leaves = [i for i, x in enumerate(is_leaves) if x]
    leaves = [str(l) for l in leaves]
    leaf_pairs = list(itertools.combinations(leaves,2))
    #print(type(tree1))
    uMG = dt_to_mg(tree1)
    res = []
    for inds in leaf_pairs:
        source = inds[0]
        target = inds[1]
        distance = nx.shortest_path_length(uMG, source=inds[0], target=inds[1])
        res.append([(source+','+target),distance])
    results = pd.DataFrame(res,columns = ['key','dist'])

    leaf_id = tree1.apply(X)
    instance_pairs = list(itertools.combinations(range(0,len(leaf_id)),2))
    instance_dist = pd.DataFrame(instance_pairs,columns = ['i1','i2'])
    instance_dist['l1'] = leaf_id[instance_dist['i1'].values]
    instance_dist['l2'] = leaf_id[instance_dist['i2'].values]
    instance_dist['leaf1'] = instance_dist[['l1','l2']].min(axis = 1)
    instance_dist['leaf2'] = instance_dist[['l1','l2']].max(axis = 1)
    instance_dist = instance_dist.drop(['l1','l2'], axis = 1)
    instance_dist['key'] = instance_dist['leaf1'].astype(str) + ',' + instance_dist['leaf2'].astype(str)
    all_distances = pd.merge(instance_dist,results,on = 'key', how = 'left')
    all_distances = all_distances.fillna(0)

    return all_distances[['i1','i2','dist']], len(leaves)

def forest_distances(rf,X):
    alldist = pd.DataFrame()
    total_nodes = 0
    for tree1 in rf.estimators_:
        dist, num_nodes = compute_distances(tree1, X)
        total_nodes += num_nodes
        alldist = alldist.append(dist)
    alldist_agg = alldist.groupby(['i1','i2']).dist.mean().reset_index()
    
    s = max(max(alldist_agg['i1']), max(alldist_agg['i2'])) + 1
    distmatrix = np.zeros(shape = (s,s))
    for i,row in alldist_agg.iterrows():
        i1 = int(row['i1'])
        i2 = int(row['i2'])
        distmatrix[i1][i2] = row['dist']
        distmatrix[i2][i1] = row['dist']

    return distmatrix, total_nodes

def dt_to_mg(clf):
    dot_data = tree.export_graphviz(clf)
    dot_graph = pydot.graph_from_dot_data(dot_data)[0]
    MG = nx.drawing.nx_pydot.from_pydot(dot_graph)
    uMG = MG.to_undirected()
    return uMG

# Model

In [17]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset, df):
    feature = df.columns[index]
    left, right = list(), list()
    indleft, indright = list(), list()
    left_df = df[df[feature] < value]
    right_df = df[df[feature] >= value]
    for i, row in enumerate(dataset):
        if row[index] < value:
            left.append(row)
            indleft.append(i)
        else:
            right.append(row)
            indright.append(i)
    #print(indleft)
    return left, right, indleft, indright, left_df, right_df
 
# Calculate the avg distance for a split dataset
def distance_index(indleft, indright, ind_tup_dct, train_final_dist_dct):
    left_len = len(indleft)
    right_len = len(indright)
    mean_left_dist = 0
    mean_right_dist = 0
    if left_len > 0:
        resultL = ind_tup_dct[indleft[0]]
        for i in indleft[:-1]:
            resultL = resultL.intersection(ind_tup_dct[i])
        for k in resultL:
            mean_left_dist += train_final_dist_dct[k]
    if right_len > 0:
        resultR = ind_tup_dct[indright[0]]
        for j in indright[:-1]:
            resultR = resultR.intersection(ind_tup_dct[j])
        for l in resultR:
            mean_right_dist += train_final_dist_dct[l]

    
    left_avg = (mean_left_dist / left_len) * (left_len / (left_len + right_len)) if left_len > 0 else 0
    right_avg = (mean_right_dist / right_len) * (right_len / (left_len + right_len)) if right_len > 0 else 0
    
    return left_avg + right_avg

# Select the best split point for a dataset
def get_split(dataset, df, ind_tup_dct, train_final_dist_dct):
    class_values = list(set(row[0] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(1, len(dataset[0])):
        for row in dataset:
            left, right, indleft, indright, left_df, right_df = test_split(index, row[index], dataset, df)
            groups = left, right, left_df, right_df
            dist = distance_index(indleft, indright, ind_tup_dct, train_final_dist_dct)
            if dist < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], dist, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}
# Create a terminal node value

def to_terminal(group):
    outcomes = [row[0] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth, ind_tup_dct, train_final_dist_dct):
    left, right, left_df, right_df = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        #print('here')
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, left_df, ind_tup_dct, train_final_dist_dct)
        split(node['left'], max_depth, min_size, depth+1, ind_tup_dct, train_final_dist_dct)
    # process right child
    if len(right) <= min_size:
        #print('here')
        node['right'] = to_terminal(right)
    else:
        #print('here')
        node['right'] = get_split(right, right_df, ind_tup_dct, train_final_dist_dct)
        split(node['right'], max_depth, min_size, depth+1, ind_tup_dct, train_final_dist_dct)
# Build a decision tree
def build_tree(train, df, max_depth, min_size, ind_tup_dct, train_final_dist_dct):
    root = get_split(train, df, ind_tup_dct, train_final_dist_dct)
    split(root, max_depth, min_size, 1, ind_tup_dct, train_final_dist_dct)
    return root
 
# Print a decision tree
def print_tree(node, df, depth=0):
    if isinstance(node, dict):
        print('%s[%s < %.3f]' % ((depth*' ', (df.columns[node['index']]), node['value'])))
        print_tree(node['left'], df, depth+1)
        print_tree(node['right'], df, depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))
        
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [18]:
def get_pred_lst(tree_full, dataset_test):
    y_pred_dt = []
    for row in dataset_test:
        y_pred_dt.append(predict(tree_full, row))
    return y_pred_dt

# Testing
Defne your own pre_process() function to return the dataframes: xTrain, xTest, yTrain, yTest, df_train, and df_test

In [19]:
num_rows_lst = []
num_dim_lst = []
depth_lst = []
time_lst = []
num_nodes = []
mean_nodes = []

In [20]:
preprocs = [pre_processing_adult(), pre_processing_adult2(), pre_processing_adult3(), pre_processing_adult4(), pre_processing_adult5(), pre_processing_adult6()]

In [23]:
def get_tree_clust(depth, pre_process_func):
    xTrain, xTest, yTrain, yTest, df_train, df_test = pre_process_func
    dataset_train = df_train.to_numpy()
    dataset_test = df_test.to_numpy()
    rf = RandomForestClassifier(n_estimators = 20, max_depth = depth).fit(xTrain,yTrain)
    distances, total_nodes = forest_distances(rf,xTrain)
    num_nodes.append(total_nodes)
    inds = [i for i in range(0, len(distances))] 
    tuples = list(product(inds, inds))
    dist = [j for sub in distances for j in sub]
    train_final_dist_dct = dict(zip(tuples, dist)) 
    ind_tup_dct = {}
    for i, j in tuples:
        if i in ind_tup_dct.keys():
            ind_tup_dct[i].add((i, j))
        else:
            ind_tup_dct[i] = set()
            ind_tup_dct[i].add((i, j))
        if j in ind_tup_dct.keys():
            ind_tup_dct[j].add((i, j))
        else:
            ind_tup_dct[j] = set()
            ind_tup_dct[j].add((i, j))
    tree = build_tree(dataset_train, df_train, depth, 30, ind_tup_dct, train_final_dist_dct)
    #ypred_dt = get_pred_lst(tree, dataset_test)
    #score = sklearn.metrics.roc_auc_score(yTest,ypred_dt)
    #print_tree(tree, df_train)
    return tree

In [ ]:
for fn in preprocs:
    print("New Dataset")
    num_rows = fn[0].shape[0] + fn[1].shape[0]
    num_dim = fn[0].shape[1]
    
    for depth in range(1, 6):
        print("new depth")
        start = time.time()
        tree_full_test_1 = get_tree_clust(depth, fn)
        end = time.time()
        num_rows_lst.append(num_rows)
        num_dim_lst.append(num_dim)
        depth_lst.append(depth)
        time_lst.append(end - start)
    

New Dataset
new depth
new depth
new depth
new depth
new depth
New Dataset
new depth
new depth
new depth
new depth
new depth


In [ ]:
df = pd.DataFrame(columns=['num_rows', 'num_cols', 'tree_depth', 'num_nodes", ''time'])
df['num_rows'] = num_rows_lst
df['num_cols'] = num_dim_lst
df['tree_depth'] = depth_lst
df['num_nodes'] = num_nodes
df['time'] = time_lst

In [ ]:
df.to_csv('test.csv')

In [ ]:
# best python debugging tool - python debugger effiecny (code profiler)
# num rep nodes vs. time (also depth/num rows vs. rep nodes)
# 

In [22]:
xTrain, xTest, yTrain, yTest, df_train, df_test = pre_processing()
start = time.time()
rf = RandomForestClassifier(n_estimators = 20, max_depth = 5).fit(xTrain,yTrain)
distances = forest_distances(rf,xTrain)
end = time.time()
print("Distance Time: "+ str(end-start))

NameError: name 'pre_processing' is not defined

In [19]:
xTrain, xTest, yTrain, yTest, df_train, df_test = pre_processing_adult2()
start = time.time()
rf = RandomForestClassifier(n_estimators = 20, max_depth = 5).fit(xTrain,yTrain)
distances = forest_distances(rf,xTrain)
end = time.time()
print("Distance Time: "+ str(end-start))

start2 = time.time()
inds = [i for i in range(0, len(distances))] 
tuples = list(product(inds, inds))
dist = [j for sub in distances for j in sub]
train_final_dist_dct = dict(zip(tuples, dist)) 
ind_tup_dct = {}
for i, j in tuples:
    if i in ind_tup_dct.keys():
        ind_tup_dct[i].add((i, j))
    else:
        ind_tup_dct[i] = set()
        ind_tup_dct[i].add((i, j))
    if j in ind_tup_dct.keys():
        ind_tup_dct[j].add((i, j))
    else:
        ind_tup_dct[j] = set()
        ind_tup_dct[j].add((i, j))

dataset_train = df_train.to_numpy()
dataset_test = df_test.to_numpy()
tree_full_test_1 = build_tree(dataset_train, df_train, 5, 30, ind_tup_dct, train_final_dist_dct)
end2 = time.time()
print("Cluster Time: "+ str(end2-start2))

KeyboardInterrupt: 

In [ ]:
xTrain, xTest, yTrain, yTest, df_train, df_test = pre_processing_adult3()
start = time.time()
rf = RandomForestClassifier(n_estimators = 20, max_depth = 5).fit(xTrain,yTrain)
distances = forest_distances(rf,xTrain)
end = time.time()
print("Distance Time: "+ str(end-start))

start2 = time.time()
inds = [i for i in range(0, len(distances))] 
tuples = list(product(inds, inds))
dist = [j for sub in distances for j in sub]
train_final_dist_dct = dict(zip(tuples, dist)) 
ind_tup_dct = {}
for i, j in tuples:
    if i in ind_tup_dct.keys():
        ind_tup_dct[i].add((i, j))
    else:
        ind_tup_dct[i] = set()
        ind_tup_dct[i].add((i, j))
    if j in ind_tup_dct.keys():
        ind_tup_dct[j].add((i, j))
    else:
        ind_tup_dct[j] = set()
        ind_tup_dct[j].add((i, j))

dataset_train = df_train.to_numpy()
dataset_test = df_test.to_numpy()
tree_full_test_1 = build_tree(dataset_train, df_train, 5, 30, ind_tup_dct, train_final_dist_dct)
end2 = time.time()
print("Cluster Time: "+ str(end2-start2))

In [ ]:
xTrain, xTest, yTrain, yTest, df_train, df_test = pre_processing_adult4()
start = time.time()
rf = RandomForestClassifier(n_estimators = 20, max_depth = 5).fit(xTrain,yTrain)
distances = forest_distances(rf,xTrain)
end = time.time()
print("Distance Time: "+ str(end-start))

start2 = time.time()
inds = [i for i in range(0, len(distances))] 
tuples = list(product(inds, inds))
dist = [j for sub in distances for j in sub]
train_final_dist_dct = dict(zip(tuples, dist)) 
ind_tup_dct = {}
for i, j in tuples:
    if i in ind_tup_dct.keys():
        ind_tup_dct[i].add((i, j))
    else:
        ind_tup_dct[i] = set()
        ind_tup_dct[i].add((i, j))
    if j in ind_tup_dct.keys():
        ind_tup_dct[j].add((i, j))
    else:
        ind_tup_dct[j] = set()
        ind_tup_dct[j].add((i, j))

dataset_train = df_train.to_numpy()
dataset_test = df_test.to_numpy()
tree_full_test_1 = build_tree(dataset_train, df_train, 5, 30, ind_tup_dct, train_final_dist_dct)
end2 = time.time()
print("Cluster Time: "+ str(end2-start2))

KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.lib.c_is_list_like'
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/abc.py", line 137, in __instancecheck__
    def __instancecheck__(cls, instance):
KeyboardInterrupt


Distance Time: 35946.87841510773


In [ ]:
xTrain, xTest, yTrain, yTest, df_train, df_test = pre_processing_adult5()
start = time.time()
rf = RandomForestClassifier(n_estimators = 20, max_depth = 5).fit(xTrain,yTrain)
distances = forest_distances(rf,xTrain)
end = time.time()
print("Distance Time: "+ str(end-start))

start2 = time.time()
inds = [i for i in range(0, len(distances))] 
tuples = list(product(inds, inds))
dist = [j for sub in distances for j in sub]
train_final_dist_dct = dict(zip(tuples, dist)) 
ind_tup_dct = {}
for i, j in tuples:
    if i in ind_tup_dct.keys():
        ind_tup_dct[i].add((i, j))
    else:
        ind_tup_dct[i] = set()
        ind_tup_dct[i].add((i, j))
    if j in ind_tup_dct.keys():
        ind_tup_dct[j].add((i, j))
    else:
        ind_tup_dct[j] = set()
        ind_tup_dct[j].add((i, j))

dataset_train = df_train.to_numpy()
dataset_test = df_test.to_numpy()
tree_full_test_1 = build_tree(dataset_train, df_train, 5, 30, ind_tup_dct, train_final_dist_dct)
end2 = time.time()
print("Cluster Time: "+ str(end2-start2))

In [ ]:
xTrain, xTest, yTrain, yTest, df_train, df_test = pre_processing_adult6()
start = time.time()
rf = RandomForestClassifier(n_estimators = 20, max_depth = 5).fit(xTrain,yTrain)
distances = forest_distances(rf,xTrain)
end = time.time()
print("Distance Time: "+ str(end-start))

start2 = time.time()
inds = [i for i in range(0, len(distances))] 
tuples = list(product(inds, inds))
dist = [j for sub in distances for j in sub]
train_final_dist_dct = dict(zip(tuples, dist)) 
ind_tup_dct = {}
for i, j in tuples:
    if i in ind_tup_dct.keys():
        ind_tup_dct[i].add((i, j))
    else:
        ind_tup_dct[i] = set()
        ind_tup_dct[i].add((i, j))
    if j in ind_tup_dct.keys():
        ind_tup_dct[j].add((i, j))
    else:
        ind_tup_dct[j] = set()
        ind_tup_dct[j].add((i, j))

dataset_train = df_train.to_numpy()
dataset_test = df_test.to_numpy()
tree_full_test_1 = build_tree(dataset_train, df_train, 5, 30, ind_tup_dct, train_final_dist_dct)
end2 = time.time()
print("Cluster Time: "+ str(end2-start2))